1. Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.

In [1]:
from __future__ import print_function
import keras # расскоментируйте эту строку, чтобы начать обучение
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [2]:
# установка параметров нейросети
batch_size = 32
num_classes = 10
num_predictions = 20

In [3]:
# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [4]:
# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [8]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [9]:
# инициализация RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

### Протестирую без data_augmentation:

In [10]:
data_augmentation = False

In [18]:
%%time
epochs = 1
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Не используется data augmentation
1563/1563 [==============================] - 172s 110ms/step - loss: 1.5299 - accuracy: 0.4405 - val_loss: 1.3895 - val_accuracy: 0.4976
сохранить обученную модель как C:\Users\Рустам\Downloads\Neural networks\saved_models\keras_cifar10_trained_model.h5 
313/313 [==============================] - 7s 23ms/step - loss: 1.3895 - accuracy: 0.4976
Test loss: 1.389495611190796
Test accuracy: 0.4975999891757965
Wall time: 3min 1s


In [20]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 7s 23ms/step - loss: 1.3895 - accuracy: 0.4976
Test loss: 1.389495611190796
Test accuracy: 0.4975999891757965


#### Видимо, качество генерации доп.изображений не очень хорошее. Без нее точность выросла до 0.49

### Протестирую большее кол-во эпох:

In [21]:
%%time
epochs = 3
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

Epoch 1/3
1563/1563 [==============================] - 174s 111ms/step - loss: 1.3815 - accuracy: 0.5035 - val_loss: 1.2738 - val_accuracy: 0.5401
Epoch 2/3
1563/1563 [==============================] - 167s 107ms/step - loss: 1.2875 - accuracy: 0.5394 - val_loss: 1.2775 - val_accuracy: 0.5617
Epoch 3/3
1563/1563 [==============================] - 176s 112ms/step - loss: 1.2099 - accuracy: 0.5696 - val_loss: 1.1714 - val_accuracy: 0.5930
Wall time: 8min 38s


In [22]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 7s 24ms/step - loss: 1.1714 - accuracy: 0.5930
Test loss: 1.1714253425598145
Test accuracy: 0.5929999947547913


#### Увеличение эпох привело к существенному росту точности. Думаю, что здесь, как и в простых нейронных сетях кол-во эпох до определенного момента повышает метрики качества сети.

### Протестирую другое окно для свертки:

In [23]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (4, 4), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 4), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [24]:
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [25]:
%%time
epochs = 3
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

Epoch 1/3
1563/1563 [==============================] - 222s 142ms/step - loss: 1.8172 - accuracy: 0.3317 - val_loss: 1.5080 - val_accuracy: 0.4534
Epoch 2/3
1563/1563 [==============================] - 231s 148ms/step - loss: 1.4934 - accuracy: 0.4601 - val_loss: 1.3338 - val_accuracy: 0.5220
Epoch 3/3
1563/1563 [==============================] - 229s 146ms/step - loss: 1.3553 - accuracy: 0.5133 - val_loss: 1.3326 - val_accuracy: 0.5249
Wall time: 11min 24s


In [26]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 9s 29ms/step - loss: 1.3326 - accuracy: 0.5249
Test loss: 1.3325718641281128
Test accuracy: 0.5249000191688538


#### Увеличение размера матрицы-окна привело к ухудшению метрики качества - видимо сглаживание идет слишком сильное

### Протестирую увеличение кол-ва слоев с увеличенным размером окна для свертки:

In [6]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (4, 4), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 4), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 4), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [11]:
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [12]:
%%time
epochs = 3
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

Epoch 1/3
1563/1563 [==============================] - 101s 65ms/step - loss: 1.8300 - accuracy: 0.3295 - val_loss: 1.5376 - val_accuracy: 0.4387
Epoch 2/3
1563/1563 [==============================] - 109s 70ms/step - loss: 1.5036 - accuracy: 0.4559 - val_loss: 1.4049 - val_accuracy: 0.5017
Epoch 3/3
1563/1563 [==============================] - 106s 68ms/step - loss: 1.3581 - accuracy: 0.5151 - val_loss: 1.2646 - val_accuracy: 0.5585
Wall time: 5min 18s


In [13]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 4s 13ms/step - loss: 1.2646 - accuracy: 0.5585
Test loss: 1.2645697593688965
Test accuracy: 0.5584999918937683


#### Как и следовало ожидать - увеличение слоев привело к улучшению результата даже с большим размером окна(т.е. более сглаженным изображением). Очевидно изначального слоя не достаточно при таком кол-ве эпох для обработки всех деталей изображений. 

#### До определенных пределов имеет смысл увеличивать кол-во эпох, далее - кол-во слоев. Окно лучше оставить 3х3 для данного датасета

2. Описать также в анализе какие необоходимо внести изменения  в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.

Для MNIST, т.к. изображения цифр черно-белые и меньшего размера (28х28), то скорей всего нужно уменьшить окно до 2х2 и можно использовать меньшее кол-во слоев и нейронов при аналогичном кол-ве эпох. 

Для CIFAR100 потребуются аналогичные настройки кол-ва эпох и слоев, но большее кол-во нейронов в слоях и большее кол-во нейронов на выходе (num_classes = 100)

Соответственно, для IMAGENET необходимо будет гораздо больше слоев, эпох и, возможно, кол-во нейронов. Также, поскольку размер изображений там 428х418 пикселей, то размер скользящего окна может быть увеличен. Кол-во нейронов на выходе должно соответствовать кол-ву определяемых классов.